In [11]:
# Instalamos Los paquetes necesarios.
!sudo apt install tesseract-ocr
!pip install pytesseract
!apt-get install poppler-utils
!pip install pdf2image
!apt-get install -y libpoppler-cpp-dev
!pip install --upgrade pdf2image
!pip install python-Levenshtein
!pip install pypdf
! wget https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa_old.traineddata #https://stackoverflow.com/questions/57968216/google-colab-how-do-i-install-traineddata-file-for-pytesseract
! sudo mv "/content/spa_old.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"
! wget https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa.traineddata #https://stackoverflow.com/questions/57968216/google-colab-how-do-i-install-traineddata-file-for-pytesseract
! sudo mv "/content/spa.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libpoppler-cpp-dev is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
--2024-12-15 20:20:14--  https://raw.githubusercontent.com/tesseract-ocr/tessdata_best/master/spa_old.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

In [12]:
import pandas as pd
import numpy as np
import sys
import random
import pypdf
from tqdm import tqdm
import os
import time
import Levenshtein
from nltk.tokenize import word_tokenize
import pytesseract
from pdf2image import convert_from_path
from pypdf import PdfReader
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
def read_words(path):
    """
    Esta funcion lee las palabras que se encuentran en la primera hoja de un excel y crea un dataframe con ellas y el count a cero.
    """
    if path.endswith('.csv'):
        df = pd.read_csv(path,sep=',')
        words = df.values.tolist()
        words = [word[0] for word in words]
    elif path.endswith('.xlsx'):
        df = pd.read_excel(path, sheet_name = 0)
        words = df['Unnamed: 0'].tolist()
        #print(df)
    elif path.endswith('.txt'):
        with open(path, 'r') as file:
            words = file.readlines()
            # replace , and ; and . with \n
            words = [word.replace(',','\n').replace(';','\n').replace('.','\n') for word in words]
            # split by \n
            words = [word.split('\n') for word in words]
            # flatten the list
            words = [item for sublist in words for item in sublist]
            # remove empty strings
            words = [word for word in words if word != '']

    else:
        print ('El archivo debe ser un csv, un excel o un txt.')
        return 1
        #raise ValueError("El archivo debe ser un csv, un excel o un txt.")

    for i in range(len(words)):
        words[i] = words[i].lower().replace(',','').replace('.','').replace(' ','')

    count = np.zeros(len(words))

    df_words = pd.DataFrame({'words':words, 'count':count})
    return df_words


def extract_images_from_pdf(pdf_path, start_page, end_page, poppler_path = None):
    """
    Extract images from a PDF file.

    Parameters:
    - pdf_path (str): Path to the PDF file.
    - start_page (int): Starting page number.
    - end_page (int): Ending page number.

    Returns:
    - images (list): List of images.
    """
    images = convert_from_path(pdf_path, dpi = 500, first_page = start_page, last_page = end_page,poppler_path = poppler_path) # poppler path is the path to the poppler bin folder
    return images



def preprocess_images(list_of_images):
    """
    Preprocess a list of images for OCR

    Parameters:
    - list_of_images (list): List of images.

    Returns:
    - images (list): List of images.
    """

    # convert images to grayscale

    images = []
    for image in list_of_images:
        image = image.convert('L')
        images.append(image)
    return images


def find_substring_with_distance(input_string, target_word, max_distance):
    # This function finds the substring of input_string that has the minimum distance to target_word
    min_distance = float('inf')
    found_substring = None

    for start in range(len(input_string)):
        for end in range(start + 1, len(input_string) + 1):
            substring = input_string[start:end]
            current_distance = levenshtein_distance(substring, target_word)

            if current_distance < min_distance and current_distance <= max_distance:
                min_distance = current_distance
                found_substring = substring

    return found_substring


def perform_ocr(image_path,language = 'spa_old'):
    text = pytesseract.image_to_string(image_path,lang = language)
    return text


def levenshtein_distance(word1, word2):
    return Levenshtein.distance(word1, word2)

def count_words_with_levenshtein(words, target_word, max_distance=2,return_coincidences=False):

    count = 0
    coincidences = []
    for word in words:
        word = find_substring_with_distance(word, target_word.lower(), max_distance)
        if word is not None:
            count += 1
            coincidences.append(word)

    if return_coincidences:
        return count, coincidences
    else:

        return count

def preprocess_text(text):
    text = text.lower()
    text = text.replace(',',' ').replace('.',' ').replace('-\n','').replace('\n',' ')
    text = word_tokenize(text)
    return text


def preprocess_csv(path_csv):
    df = pd.read_csv(path_csv, index_col = 0)
    df = df.dropna()
    df['coincidences'] = df['coincidences'].str.split(',')
    df['Page'] = df['Page'].str.split(',')
    return df


from collections import Counter

def combinar_listas(lista_palabras, lista_paginas):
    # Crear un diccionario para almacenar las páginas asociadas a cada palabra
    diccionario_paginas = {}

    # Llenar el diccionario con las páginas asociadas a cada palabra
    for palabra, pagina in zip(lista_palabras, lista_paginas):
        if palabra in diccionario_paginas:
            diccionario_paginas[palabra]['paginas'].append(pagina)
            diccionario_paginas[palabra]['contador'] += 1
        else:
            diccionario_paginas[palabra] = {'paginas': [pagina], 'contador': 1}

    # Crear la lista combinada
    lista_combinada = [f"{palabra} ({', '.join(map(str, info['paginas']))}) [{info['contador']}]" for palabra, info in diccionario_paginas.items()]

    return lista_combinada

In [14]:


def Perform_word_count(path_pdf, path_csv, start_page = 1, end_page = -1, max_distance = 2, return_coincidences = True, path_output= None, formato = '.csv',
                       number_of_returns = 'Uno',folder = False,language = 'spa'):
    # Control de errores

    # print ('---------------------------------------------------')

    print ('Iniciando análisis de texto...')
    # print ('Parámetros:')
    print ('PDF: ', path_pdf)
    # print ('Archivo de entrada: ', path_csv)
    # print ('Página inicial: ', start_page)
    # print ('Página final: ', end_page)
    # print ('Distancia máxima: ', max_distance)
    # print ('Devolver coincidencias: ', return_coincidences)
    # print ('Archivo de salida: ', path_output)
    # print ('Formato de archivo de salida: ', formato)
    # print ('Número de retornos: ', number_of_returns)
    # print ('Modo folder?: ', folder)

    # print ('---------------------------------------------------')


    if start_page < 1:
        print ('La página inicial debe ser mayor que 0, se establece el parámetro a 1.')
        start_page = 1
    if end_page <= start_page and end_page != -1:
        print ('La página final debe ser mayor que la inicial, puede insertar -1 para indicar que es la última página del documento.')
        return 1
        #raise ValueError("La página final debe ser mayor que la inicial, puede insertar -1 para indicar que es la última página del documento.")
    if max_distance <= 0:
        print ('La distancia máxima debe ser mayor que 0.')
        return 1
        #raise ValueError("La distancia máxima debe ser mayor que 0.")
    if path_output != None:
        if formato not in ['.csv','.xlsx','.txt']:
            print ('El formato de archivo de salida no es válido, se establece el formato a .csv')
            formato = '.csv'

    # calculate the number of pages of the pdf
    with open(path_pdf, 'rb') as file:
        pdf_reader = PdfReader(file)
        num_pages = len(pdf_reader.pages)

    if end_page > num_pages or end_page == -1:
        print ('La página final debe ser menor que el número de páginas del pdf, que es {}, se establece el parámetro a este último.'.format(num_pages))
        end_page = num_pages

    print ('Número de páginas del PDF: ', num_pages)
    print ('Leyendo desde la página {} hasta la página {}'.format(start_page,end_page))
    #print ('Poppler path: ', poppler_path)
    # extract images from pdf

    try:
        images = convert_from_path(path_pdf, size = 800, first_page = start_page, last_page=end_page,poppler_path='/usr/bin')
    except Exception as e:
        #print the error
        print (e)
        print ('Error al extraer las imágenes del PDF.')
        return 1
        #raise ValueError("Error al extraer las imágenes del PDF.")
    images = preprocess_images(images)
    text = ''

    print ('Digitalizando el texto...')
    print ('Esto puede tardar unos minutos, por favor, espere.')
    page_counter = start_page
    book = []
    #print ('tesseract path: ', tesseract_path)
    for image in tqdm(images):
        try:
            text = perform_ocr(image,language=language)
        except Exception as e:
            #print the error
            print (e)
            print ('Error al realizar OCR.')
            return 1
            #raise ValueError("Error al realizar OCR.")
        book.append([page_counter,text])
        page_counter += 1

    print('Buscando palabras...')

    try:
        df_words = read_words(path_csv)
    except:
        print ('Error al leer el archivo de entrada de palabras.')
        return 1
        #raise ValueError("Error al leer el archivo de entrada.")

    for i in range(len(book)):
        book[i][1] = preprocess_text(book[i][1])


    if return_coincidences:
        df_words['coincidences'] = np.nan
        df_words['Page'] = np.nan

    for index, row in df_words.iterrows():
        if return_coincidences:
            counter = 0
            coincidences = []
            page_numbers = []
            for page in book:
                count,coincidence = count_words_with_levenshtein(page[1], row['words'], max_distance,return_coincidences)
                counter += count
                coincidences.extend(coincidence)
                page_numbers.extend([page[0]]*len(coincidence))


            coincidences = np.array(coincidences)
                # Convertir el array de coincidencias en un string
            coincidences = ','.join(coincidences)
                # return the page where the word was found
            page_numbers = np.array(page_numbers)
            page_numbers = page_numbers.astype(str)
            page_numbers = ','.join(page_numbers)

            df_words.loc[index, 'Page'] = page_numbers
            df_words.loc[index, 'count'] = counter
            df_words.loc[index, 'coincidences'] = coincidences

        else:

            for page in book:
                count = 0
                count = count_words_with_levenshtein(page[1], row['words'], max_distance,return_coincidences)
                count += count
            df_words.loc[index, 'count'] = count


    print ('Guardando resultados...')
    df_words = df_words.dropna()
    df_new = pd.DataFrame(columns = ['Palabra','Coincidencias exactas','Coincidencias no exactas'])
    for index, row in df_words.iterrows():
        non_exact = []
        coincidentes = row['coincidences']
        coincidentes = coincidentes.split(',')
        pages = row['Page']
        pages = pages.split(',')

        coincidencias = combinar_listas(coincidentes,pages)

        df_new.loc[index,'Palabra'] = row['words']

        for coincidencia in coincidencias:
            objetivo = df_words.loc[index, 'words']
            #print ('Objetivo:',objetivo)
            #print (coincidencias)

            palabra = coincidencia.split(' ')[0]
            if levenshtein_distance(palabra, objetivo) == 0:
                df_new.loc[index,'Coincidencias exactas'] = coincidencia

            else:
                non_exact.append(coincidencia)

        if number_of_returns == 'Todas':
            df_new.loc[index,'Coincidencias no exactas'] = non_exact
        elif number_of_returns == 'Uno':
            if len(non_exact) > 0:
                df_new.loc[index,'Coincidencias no exactas'] = random.choice(non_exact)
            else:
                df_new.loc[index,'Coincidencias no exactas'] = ''
        elif number_of_returns == 'Dos':
            if len(non_exact) > 0:
                if len(non_exact) < 2:
                    df_new.loc[index,'Coincidencias no exactas'] = non_exact
                else:
                    aux = []
                    aux.append(random.choice(non_exact))
                    aux.append(random.choice(non_exact))
                    df_new.loc[index,'Coincidencias no exactas'] = aux


            else:
                df_new.loc[index,'Coincidencias no exactas'] = ''

        else:
            # Else es Ninguno
            df_new.loc[index,'Coincidencias no exactas'] = ''




    # if you put the format also in the name of the file, delete it.add()

    if path_output != None:
        if '.' in path_output:
            path_output = path_output.split('.')[:-1]
            path_output = '.'.join(path_output)
            path_output = path_output
    else:
        path_output = 'output'

    # if we are perfoming the analysis for a folder, we add the name of the pdf to the output file, to avoid overwriting

    if folder:
        path_output = path_output+'_'+ path_pdf.split('\\')[-1].split('.')[0]

    # save the file in the desired format

    if formato == '.csv':
        df_new.to_csv(path_output + '.csv')

    if formato == '.xlsx':
        df_new.to_excel(path_output+'.xlsx')

    if formato == '.txt':
        df_new.to_csv(path_output+ '.txt',sep='\t')

    print ('Archivo guardado en: ', path_output + formato)



In [15]:
name_excel = 'antillanismos.xlsx'
name_pdf = '1637-Moreno.pdf'
path_output = 'Output.xlsx'

path_csv = '/content/' + name_excel
path_pdf = '/content/' + name_pdf

start_page = 1
end_page = -1
max_distance = 2
return_coincidences = True
formato = '.xlsx'
number_of_returns = 2
folder = False



Perform_word_count(path_pdf, path_csv, start_page, end_page, max_distance, return_coincidences, path_output, formato,
                       number_of_returns, folder,'spa')


Iniciando análisis de texto...
PDF:  /content/1637-Moreno.pdf
La página final debe ser menor que el número de páginas del pdf, que es 73, se establece el parámetro a este último.
Número de páginas del PDF:  73
Leyendo desde la página 1 hasta la página 73
Digitalizando el texto...
Esto puede tardar unos minutos, por favor, espere.


100%|██████████| 73/73 [14:01<00:00, 11.53s/it]


Buscando palabras...


<ipython-input-14-6c2c44de1284>:118: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3,3,3,3,4,4,4,6,7,7,7,7,7,7,7,7,7,8,8,8,8,8,9,9,9,9,9,9,9,10,10,10,10,10,10,10,10,11,11,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,12,12,12,12,13,13,13,13,13,13,13,13,14,14,14,14,14,14,14,14,14,14,14,14,14,15,15,15,15,15,16,16,17,17,17,17,17,17,17,17,17,17,17,17,17,18,18,18,18,18,19,19,19,19,19,19,19,19,19,19,19,19,19,19,20,20,20,20,20,20,20,20,20,20,20,20,21,21,21,21,21,21,21,21,21,21,21,21,21,21,22,22,22,22,22,22,22,22,22,22,23,23,23,23,23,23,23,23,23,23,23,23,24,24,24,24,25,25,25,25,25,26,26,26,26,26,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,28,28,28,28,28,28,28,28,28,29,29,29,29,29,29,29,29,29,29,30,30,30,30,30,30,30,30,30,30,31,31,31,31,31,31,31,31,31,31,31,31,31,32,32,32,32,32,32,32,32,32,32,32,32,33,33,33,33,33,33,33,33,33,33,34,34,34,34,34,34,34,34,34,34,34,35,35,35,35,35,35,35,35,35,35,35,36,36,36,36

Guardando resultados...
Archivo guardado en:  Output.xlsx
